In [225]:
import os 
import numpy as np 
import pandas as pd 
from pandas import Series, DataFrame 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import OneHotEncoder 
from sklearn import svm
from sklearn.datasets import samples_generator
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVR
import matplotlib.pyplot as plt
import ssl 
import pymysql
import os
from sklearn.externals import joblib

# used to fix Python SSL CERTIFICATE_VERIFY_FAILED
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context

In [226]:
#Load data as we need to do min-max normalization
project_data_path = '/Users/charles/Desktop/Project/LongTan_File/'
longtan_file_2018 = os.path.join(project_data_path, 'LongTan_complete_2018.csv')
print('Path of read in data: %s' % (longtan_file_2018))

Path of read in data: /Users/charles/Desktop/Project/LongTan_File/LongTan_complete_2018.csv


In [227]:
project_data = pd.read_csv(longtan_file_2018)

In [228]:
#Restore model
clf=joblib.load("./LT_ML_Predict_V2.pkl")

In [229]:
#Connect MySQL
db = pymysql.connect(host='localhost', port=3306, user='root', db='Air_pollution', charset='utf8')
cursor = db.cursor()

cursor.execute("SELECT Station,Year,Month,Day,Time,SO2,CO,O3,PM10,NOx,NO,NO2,WIND_SPEED,WIND_DIREC,AMB_TEMP,RAINFALL,RH,WS_HR,WD_HR,PM25 FROM LongTan_real_time_2020 ORDER BY ID DESC LIMIT 1")
#cursor.execute("")

#Fetch data
LongTan_real_time_2020 = cursor.fetchone()

#Close connection
db.close()

print(LongTan_real_time_2020)

('LongTan', 2020, 1, 7, 15, 1.2, 0.4, 55, 23, 16.0, 1.9, 14.0, 3.1, 258, 24.5, 0, 3, 54.0, 236, 17)


In [230]:
#Show type of LongTan_real_time_2020
type(LongTan_real_time_2020)

tuple

In [231]:
#Convert tuple to list
LongTan_real_time_2020 = list(LongTan_real_time_2020)

In [232]:
#Show MySQL column name
num_fields = len(cursor.description)
field_names = [i[0] for i in cursor.description]

In [233]:
#Convert list to dictionary
#Convert dictionary to dataframe
list_dict = zip(field_names, LongTan_real_time_2020)
dict_dataframe = dict(list_dict)
df_complete = pd.DataFrame([dict_dataframe ])

In [234]:
df_complete

,AMB_TEMP,CO,Day,Month,NO,NO2,NOx,O3,PM10,PM25,RAINFALL,RH,SO2,Station,Time,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR,Year
0,24.5,0.4,7,1,1.9,14.0,16.0,55,23,17,0,3,1.2,LongTan,15,236,258,3.1,54.0,2020


In [235]:
df_complete = df_complete[['Station',
 'Year',
 'Month',
 'Day',
 'Time',
 'AMB_TEMP',
 'CO',
 'NO',
 'NOx',
 'O3',
 'PM10',
 'PM25',
 'RAINFALL',
 'RH',
 'SO2',
 'WIND_SPEED',
 'WS_HR',
 'WD_HR',
 'WIND_DIREC'
  ]]

In [236]:
df_complete

,Station,Year,Month,Day,Time,AMB_TEMP,CO,NO,NOx,O3,PM10,PM25,RAINFALL,RH,SO2,WIND_SPEED,WS_HR,WD_HR,WIND_DIREC
0,LongTan,2020,1,7,15,24.5,0.4,1.9,16.0,55,23,17,0,3,1.2,3.1,54.0,236,258


In [237]:
df_complete = df_complete.rename(columns={"PM25": "PM2.5"})
df_complete

,Station,Year,Month,Day,Time,AMB_TEMP,CO,NO,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,WIND_SPEED,WS_HR,WD_HR,WIND_DIREC
0,LongTan,2020,1,7,15,24.5,0.4,1.9,16.0,55,23,17,0,3,1.2,3.1,54.0,236,258


In [238]:
df_complete['WD_HR'] = np.where(df_complete['WD_HR'].between(0,22.5),1, df_complete['WD_HR'])
df_complete['WD_HR'] = np.where(df_complete['WD_HR'].between(22.5,67.5),2, df_complete['WD_HR'])
df_complete['WD_HR'] = np.where(df_complete['WD_HR'].between(67.5,112.5),3, df_complete['WD_HR'])
df_complete['WD_HR'] = np.where(df_complete['WD_HR'].between(112.5,157.5),4, df_complete['WD_HR'])
df_complete['WD_HR'] = np.where(df_complete['WD_HR'].between(157.5,202.5),5, df_complete['WD_HR'])
df_complete['WD_HR'] = np.where(df_complete['WD_HR'].between(202.5,247.5),6, df_complete['WD_HR'])
df_complete['WD_HR'] = np.where(df_complete['WD_HR'].between(247.5,292.5),7, df_complete['WD_HR'])
df_complete['WD_HR'] = np.where(df_complete['WD_HR'].between(292.5,337.5),8, df_complete['WD_HR'])
df_complete['WD_HR'] = df_complete['WD_HR'].mask(df_complete['WD_HR'] > 337.5, 1)

In [239]:
#Convert WIND_DIREC to 8 degree
df_complete['WIND_DIREC'] = np.where(df_complete['WIND_DIREC'].between(0,22.5),1, df_complete['WIND_DIREC'])
df_complete['WIND_DIREC'] = np.where(df_complete['WIND_DIREC'].between(22.5,67.5),2, df_complete['WIND_DIREC'])
df_complete['WIND_DIREC'] = np.where(df_complete['WIND_DIREC'].between(67.5,112.5),3, df_complete['WIND_DIREC'])
df_complete['WIND_DIREC'] = np.where(df_complete['WIND_DIREC'].between(112.5,157.5),4, df_complete['WIND_DIREC'])
df_complete['WIND_DIREC'] = np.where(df_complete['WIND_DIREC'].between(157.5,202.5),5, df_complete['WIND_DIREC'])
df_complete['WIND_DIREC'] = np.where(df_complete['WIND_DIREC'].between(202.5,247.5),6, df_complete['WIND_DIREC'])
df_complete['WIND_DIREC'] = np.where(df_complete['WIND_DIREC'].between(247.5,292.5),7, df_complete['WIND_DIREC'])
df_complete['WIND_DIREC'] = np.where(df_complete['WIND_DIREC'].between(292.5,337.5),8, df_complete['WIND_DIREC'])
df_complete['WIND_DIREC'] = df_complete['WIND_DIREC'].mask(df_complete['WIND_DIREC'] > 337.5, 1)

In [240]:
#Convert WIND_SPEED to 4 degree
df_complete['WIND_SPEED'] = np.where(df_complete['WIND_SPEED'].between(0,0.3),1, df_complete['WIND_SPEED'])
df_complete['WIND_SPEED'] = np.where(df_complete['WIND_SPEED'].between(0.3,3.4),2, df_complete['WIND_SPEED'])
df_complete['WIND_SPEED'] = np.where(df_complete['WIND_SPEED'].between(3.4,8),3, df_complete['WIND_SPEED'])
df_complete['WIND_SPEED'] = df_complete['WIND_SPEED'].mask(df_complete['WIND_SPEED'] > 8, 4)

In [241]:
#Convert WS_HR to 4 degree
df_complete['WS_HR'] = np.where(df_complete['WS_HR'].between(0,0.3),1, df_complete['WS_HR'])
df_complete['WS_HR'] = np.where(df_complete['WS_HR'].between(0.3,3.4),2, df_complete['WS_HR'])
df_complete['WS_HR'] = np.where(df_complete['WS_HR'].between(3.4,8),3, df_complete['WS_HR'])
df_complete['WS_HR'] = df_complete['WS_HR'].mask(df_complete['WS_HR'] > 8, 4)

In [242]:
#Convert type to integer because of one hot encoding 
df_complete['WD_HR']= df_complete['WD_HR'].round(0).astype(int)
df_complete['WIND_DIREC']= df_complete['WIND_DIREC'].round(0).astype(int)

In [243]:
#One hot encoding
df_complete_WD_HR_OHE = pd.get_dummies(df_complete['WD_HR'],prefix=['WD_HR_OHE'])
df_complete_WIND_DIREC_OHE = pd.get_dummies(df_complete['WIND_DIREC'],prefix=['WIND_DIREC_OHE'])

In [244]:
df_complete

,Station,Year,Month,Day,Time,AMB_TEMP,CO,NO,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,WIND_SPEED,WS_HR,WD_HR,WIND_DIREC
0,LongTan,2020,1,7,15,24.5,0.4,1.9,16.0,55,23,17,0,3,1.2,2.0,4.0,6,7


In [245]:
#Create a list and insert column name 
create_OHE_WD_HR_column_name = ("['WD_HR_OHE']_1",
 "['WD_HR_OHE']_2",
 "['WD_HR_OHE']_3",
 "['WD_HR_OHE']_4",
 "['WD_HR_OHE']_5",
 "['WD_HR_OHE']_6",
 "['WD_HR_OHE']_7",
 "['WD_HR_OHE']_8",
)

In [246]:
#Convert type
create_OHE_WD_HR_column_name = list(create_OHE_WD_HR_column_name)

In [247]:
#Create zero*7 list and return a list with 8 values
def display(value):
    s1 = [0]*7 
    s1.insert(value-1,1)
    return s1

In [248]:
#Fetch value to insert function
WD_HR_OHE_list_value = display(df_complete.iloc[0,17])

In [249]:
#Convert list to dictionary
#Convert dictionary to dataframe
OHE_WD_HR_list_dict = zip(create_OHE_WD_HR_column_name , WD_HR_OHE_list_value)
OHE_WD_HR_dict_dataframe = dict(OHE_WD_HR_list_dict)
OHE_WD_HR_complete = pd.DataFrame([OHE_WD_HR_dict_dataframe])

In [250]:
#Show dataframe
OHE_WD_HR_complete

,['WD_HR_OHE']_1,['WD_HR_OHE']_2,['WD_HR_OHE']_3,['WD_HR_OHE']_4,['WD_HR_OHE']_5,['WD_HR_OHE']_6,['WD_HR_OHE']_7,['WD_HR_OHE']_8
0,0,0,0,0,0,1,0,0


In [251]:
#Create a list and insert column name 
create_OHE_WIND_DIREC_column_name= ("['WIND_DIREC_OHE']_1",
 "['WIND_DIREC_OHE']_2",
 "['WIND_DIREC_OHE']_3",
 "['WIND_DIREC_OHE']_4",
 "['WIND_DIREC_OHE']_5",
 "['WIND_DIREC_OHE']_6",
 "['WIND_DIREC_OHE']_7",
 "['WIND_DIREC_OHE']_8",
)

In [252]:
#Convert type
create_OHE_WIND_DIREC_column_name = list(create_OHE_WIND_DIREC_column_name)

In [253]:
#Fetch value to insert function
WIND_DIREC_OHE_list_value = display(df_complete.iloc[0,18])

In [254]:
#Convert list to dictionary
#Convert dictionary to dataframe
OHE_WIND_DIREC_list_dict = zip(create_OHE_WIND_DIREC_column_name , WIND_DIREC_OHE_list_value)
OHE_WIND_DIREC_dict_dataframe = dict(OHE_WIND_DIREC_list_dict)
OHE_WIND_DIREC_complete = pd.DataFrame([OHE_WIND_DIREC_dict_dataframe])

In [255]:
#Show dataframe
OHE_WIND_DIREC_complete 

,['WIND_DIREC_OHE']_1,['WIND_DIREC_OHE']_2,['WIND_DIREC_OHE']_3,['WIND_DIREC_OHE']_4,['WIND_DIREC_OHE']_5,['WIND_DIREC_OHE']_6,['WIND_DIREC_OHE']_7,['WIND_DIREC_OHE']_8
0,0,0,0,0,0,0,1,0


In [256]:
df_complete = df_complete.iloc[:,5:17]
df_complete

,AMB_TEMP,CO,NO,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,WIND_SPEED,WS_HR
0,24.5,0.4,1.9,16.0,55,23,17,0,3,1.2,2.0,4.0


In [257]:
#Concatenate dataframe after doing one hot encoding in each component
Wind_OHE_df_complete = pd.concat([OHE_WD_HR_complete,OHE_WIND_DIREC_complete], axis=1)
Wind_OHE_df_complete 

,['WD_HR_OHE']_1,['WD_HR_OHE']_2,['WD_HR_OHE']_3,['WD_HR_OHE']_4,['WD_HR_OHE']_5,['WD_HR_OHE']_6,['WD_HR_OHE']_7,['WD_HR_OHE']_8,['WIND_DIREC_OHE']_1,['WIND_DIREC_OHE']_2,['WIND_DIREC_OHE']_3,['WIND_DIREC_OHE']_4,['WIND_DIREC_OHE']_5,['WIND_DIREC_OHE']_6,['WIND_DIREC_OHE']_7,['WIND_DIREC_OHE']_8
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0


In [258]:
#Select columns to insert model
new_x_dataframe= pd.concat([df_complete,Wind_OHE_df_complete], axis=1)
new_x_dataframe

,AMB_TEMP,CO,NO,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,...,['WD_HR_OHE']_7,['WD_HR_OHE']_8,['WIND_DIREC_OHE']_1,['WIND_DIREC_OHE']_2,['WIND_DIREC_OHE']_3,['WIND_DIREC_OHE']_4,['WIND_DIREC_OHE']_5,['WIND_DIREC_OHE']_6,['WIND_DIREC_OHE']_7,['WIND_DIREC_OHE']_8
0,24.5,0.4,1.9,16.0,55,23,17,0,3,1.2,...,0,0,0,0,0,0,0,0,1,0


In [259]:
#Min-Max Normalization
new_x_dataframe['AMB_TEMP'] = (new_x_dataframe['AMB_TEMP']-project_data['AMB_TEMP'].min())/(project_data['AMB_TEMP'].max()-project_data['AMB_TEMP'].min())
new_x_dataframe['CO'] = (new_x_dataframe['CO']-project_data['CO'].min())/(project_data['CO'].max()-project_data['CO'].min())
new_x_dataframe['NO'] = (new_x_dataframe['NO']-project_data['NO'].min())/(project_data['NO'].max()-project_data['NO'].min())
new_x_dataframe['NOx'] = (new_x_dataframe['NOx']-project_data['NOx'].min())/(project_data['NOx'].max()-project_data['NOx'].min())
new_x_dataframe['O3'] = (new_x_dataframe['O3']-project_data['O3'].min())/(project_data['O3'].max()-project_data['O3'].min())
new_x_dataframe['PM10'] = (new_x_dataframe['PM10']-project_data['PM10'].min())/(project_data['PM10'].max()-project_data['PM10'].min())
new_x_dataframe['RAINFALL'] = (new_x_dataframe['RAINFALL']-project_data['RAINFALL'].min())/(project_data['RAINFALL'].max()-project_data['RAINFALL'].min())
new_x_dataframe['RH'] = (new_x_dataframe['RH']-project_data['RH'].min())/(project_data['RH'].max()-project_data['RH'].min())
new_x_dataframe['SO2'] = (new_x_dataframe['SO2']-project_data['SO2'].min())/(project_data['SO2'].max()-project_data['SO2'].min())
new_x_dataframe['PM2.5'] = (new_x_dataframe['PM2.5']-project_data['PM2.5'].min())/(project_data['PM2.5'].max()-project_data['PM2.5'].min())
new_x_dataframe['WIND_SPEED'] = (new_x_dataframe['WIND_SPEED']-project_data['WIND_SPEED'].min())/(project_data['WIND_SPEED'].max()-project_data['WIND_SPEED'].min())
new_x_dataframe['WS_HR'] = (new_x_dataframe['WS_HR']-project_data['WS_HR'].min())/(project_data['WS_HR'].max()-project_data['WS_HR'].min())



In [260]:
new_x_dataframe

,AMB_TEMP,CO,NO,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,...,['WD_HR_OHE']_7,['WD_HR_OHE']_8,['WIND_DIREC_OHE']_1,['WIND_DIREC_OHE']_2,['WIND_DIREC_OHE']_3,['WIND_DIREC_OHE']_4,['WIND_DIREC_OHE']_5,['WIND_DIREC_OHE']_6,['WIND_DIREC_OHE']_7,['WIND_DIREC_OHE']_8
0,0.551633,0.496063,0.206339,0.40233,0.618574,0.239358,0.293967,0.0,-0.535853,0.216127,...,0,0,0,0,0,0,0,0,1,0


In [261]:
#Convert predict_y to list
predict_y = clf.predict(new_x_dataframe.values).tolist()

In [262]:
#Convert list to string
predict_y = ' '.join(map(str,predict_y)) 

In [263]:
#Show final prediction
print(predict_y) 

17.53837379561243


In [264]:
predict_y_redis = round(float(predict_y),2)
predict_y_redis 

17.54

In [265]:
import redis
conn = redis.StrictRedis(host="localhost", port=6379, charset="utf-8", decode_responses=True)
conn.hset('prediction','LongTan',predict_y_redis)
print(conn.hgetall('prediction'))

{'GuanYin': '25.83', 'TaoYuan': '21.09', 'PingZhen': '19.02', 'JhongLi': '27.77', 'LongTan': '17.54'}
